In [1]:
import json
from neo4j import GraphDatabase
import pandas as pd

In [2]:
#Load dataset
with open('annotated-data.json', 'r') as file:
    data = json.load(file)

In [29]:
#Use for global database connection
driver = GraphDatabase.driver(uri="bolt://3.91.134.249:7687", auth=("neo4j", "escapes-certification-array"))

In [3]:
#use for local database connection
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "password"))

In [4]:
#Convert JSON to DataFrame
df = pd.DataFrame(data)

In [12]:
df

,id,data,annotations,tid,title,doc
0,9773679d174baecd8dea3a3d04ad2eb06f8d66b68cd9a6...,"{'text': 'R.V.Raveendran, J. The claiman...","[{'id': '76b3f31eb53c4fa08ba719e2aac15329', 's...",837924,Sarla Verma & Ors vs Delhi Transport Corp.& An...,"R.V.RAVEENDRAN, J. The claimants in a mot..."
1,f97f6956004592a7f9b987c414c7180db06015860cf377...,{'text': 'J U D G M E N T W I T HSLP (C)...,"[{'id': 'dbc821fbf4e7444f95ad02fac5cd9828', 's...",1827019,National Insurance Co. Ltd vs Swaran Singh & O...,"J U D G M E N T W I T HSLP (C) 10017/03, ..."
2,fa839ebdad024476ac233e42e9bbe0c377c27f4031f1dd...,{'text': 'J U D G M E N T P.K. BALASUBRAMANYA...,"[{'id': 'daf47aad24f44ed0828f96a88442b463', 's...",1964308,The Oriental Insurance Company ... vs Meena Va...,"J U D G M E N T P.K. BALASUBRAMANYAN, J.1.One ..."
3,f242dda5ce530398b22e510ebeca07ef331e5f95313b08...,{'text': 'L.....I.........T.......T.......T......,"[{'id': 'd46f0be64a014504b6ebb28515f7eeb6', 's...",484625,Smt. Kaushnuma Begum And Ors vs The New India ...,L.....I.........T.......T.......T.......T........
4,28331be12dc40bfb74b59f77ba1b30059703cdf546e918...,"{'text': ' R.M. Lodha,J. A two-J...","[{'id': 'b1b630bb34334ddcbe8c364f0252041c', 's...",86908489,Reshma Kumari & Ors vs Madan Mohan & Anr on 2 ...,"R.M. LODHA,J. A two-Judge Bench..."
...,...,...,...,...,...,...
328,80f1e360d5e2442e74a7ca96962cc14799070a2dc91bd5...,"{'text': 'Judgment Varghese Kalliath, J.1. Thi...","[{'id': '6cbb13dbb7f84912968596510710193f', 's...",125801,Motor And General Finance (India) ... vs Mary ...,"JUDGMENT Varghese Kalliath, J.1. This is a phe..."
329,ab377c9809b20910230057a7902cefd1e017416f66584d...,"{'text': 'Judgment B.S. Kapadia, J.1. The pre...","[{'id': '1da1f4643bdc46489a9a2177934aebf0', 's...",437556,Manohar Jamatmal Sindhi And Anr. vs Ranguba An...,"JUDGMENT B.S. Kapadia, J.1. The present appeal..."
330,fc71c49e54b831d945dd59161f290f398144fb4f50bc89...,"{'text': 'Judgment K.M. Jatarajan, J.1. These...","[{'id': 'a5e2bf151b7641689539bcb23fbe3b6d', 's...",791009,"Principal, Tamil Nadu ... vs A. Saraswathi And...","JUDGMENT K.M. Jatarajan, J.1. These three civi..."
331,f152ce410cb653bb95aab27d589511083a032136198250...,"{'text': 'Judgment A.M. Ahkadi, J. 1. Mr. H....","[{'id': 'ef3726f11cf94394a184ed24e8f1530a', 's...",934048,Prataprai Arjandas Dhameja And ... vs Bhupatsi...,"JUDGMENT A.M. Ahkadi, J. 1. Mr. H. P. Shroff ..."


In [6]:
df.columns

Index(['id', 'data', 'annotations', 'tid', 'title', 'doc'], dtype='object')

In [13]:
#Method to print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total:
        print()

In [144]:
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

In [145]:
#Add case nodes to database
count = 0
total = len(df)
print("Total :", len(df))
print("CREATING GRAPH: ")
printProgressBar(0, total, prefix='Progress:', suffix='Complete', length=total)

with driver.session() as session:

    for index, row in df.iterrows():

        doc = row['doc']
        tid = row['tid']
        annotation_id = row['id']
        title = row['title']

        if session.run(
        "MATCH (n:Case {name: $name, case_id: $case_id}) RETURN n",
         name=title, case_id=tid).single() is None:

            #Creating case nodes
            a = session.run("CREATE (n:Case {name: $name, case_id: $case_id, document: $document, annotation_id: $annotation_id})", name=title, case_id=tid, document=doc, annotation_id=annotation_id)

        annotations = pd.DataFrame(row['annotations'])

        for index_annotations, row_annotations in annotations.iterrows():

            text_id = row_annotations['id']
            text = row_annotations['text']

            entities = pd.DataFrame(row_annotations['entities'])
            if len(entities) != 0:

                #checking if sub-text nodes exist
                if session.run(
                    "MATCH (n:SubText {name: $text, text_id: $text_id, case_annotation_id: $case_annotation_id}) RETURN n",
                    text=text, text_id=text_id, case_annotation_id=annotation_id).single() is None:

                    #Creating case sub-text nodes
                    session.run("CREATE (n:SubText {name: $text, text_id: $text_id, case_annotation_id: $case_annotation_id})",
                                text=text, text_id=text_id, case_annotation_id=annotation_id)

                    #Connecting sub-text nodes to case nodes
                    session.run("MATCH (a:SubText),(b:Case) "
                                "WHERE a.case_annotation_id = $case_annotation_id "
                                "AND b.annotation_id = $case_annotation_id "
                                "AND a.text_id = $text_id "
                                "MERGE (a)-[ r:SUBTEXT_OF {relationship: $relationship}]->(b)",
                                case_annotation_id=annotation_id,
                                relationship="Sub-Text with ID: " + str(text_id) + ", of case with ID " + str(tid), text_id=text_id)

                #TODO: Create node for entities and connect them to all sentences, matching entities of anything above 0.6 should be connected to all ndoes
                for index_entities, row_entities in entities.iterrows():

                    normalized_name = row_entities['normalized_name']
                    labels = row_entities['labels']

                    for label in labels:
                        if session.run("MATCH (n:Entity {type: $type})"
                                       "WHERE apoc.text.levenshteinSimilarity(toLower($name), toLower(n.name)) > 0.6 "
                                       "RETURN n", name=normalized_name,
                       type=label) is None:

                            session.run("CREATE (n:Entity {name: $name, type: $type, parent_id: $parent_id})",
                                        name=normalized_name, type=label, parent_id=[text_id])

                        else:
                            session.run("MATCH (n:Entity {type: $type})"
                                        "WHERE apoc.text.levenshteinSimilarity(toLower($name), toLower(n.name)) > 0.6 "
                                        "SET n.parent_id = apoc.coll.union("
                                        "n.parent_id, $parent_id)",
                                        name=normalized_name, type=label, parent_id=[text_id])


                        relationship = "Normalized Name found in subtext: " + str(text_id) + ", of case with ID " + str(tid)

                        session.run("MATCH (a:Entity {type: $type}),"
                                    "(b:SubText {text_id: $parent_id, case_annotation_id: $case_annotation_id}) "
                                    "WHERE $parent_id IN a.parent_id  "
                                    "MERGE (a)-[ r:" + label + "_OF" + " {relationship: $relationship}]->(b)",
                                    parent_id=text_id,
                                    type=label,
                                    case_annotation_id=annotation_id,
                                    relationship=relationship,
                                    label=label)

        count = count + 1
        printProgressBar(count, total, prefix='Progress:', suffix='Complete', length=total)

Total : 333
CREATING GRAPH: 


/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "


/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a singl

/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a singl

/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a singl

/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a singl

/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/lib/python3.8/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a singl

KeyboardInterrupt: 